In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

from tqdm.keras import TqdmCallback
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import datetime

In [2]:
# Train
size = 'demo'
type_ = 'train'

numpy_file_X_path = f'./files/numpy/X_{type_}_{size}.npy'
numpy_file_y_path = f'./files/numpy/y_{type_}_{size}.npy'

X_train = np.load(numpy_file_X_path)
y_train = np.load(numpy_file_y_path)

In [3]:
# Validation 
size = 'demo'
type_ = 'validation'

numpy_file_X_path = f'./files/numpy/X_{type_}_{size}.npy'
numpy_file_y_path = f'./files/numpy/y_{type_}_{size}.npy'

X_val = np.load(numpy_file_X_path)
y_val = np.load(numpy_file_y_path)

In [4]:
# Use only the first 100 rows for testing
# X_train = X_train[:80]
# y_train = y_train[:80]
# X_val = X_val[:20]
# y_val = y_val[:20]

In [5]:
# Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

In [6]:
# Define the model
model = Sequential([
    tf.keras.Input(shape=(600,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation = 'linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])


In [7]:

# Prepare TensorBoard callback
log_dir = "files/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
num_epochs = 2
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=16,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, TqdmCallback(verbose=1)],
                    verbose=2)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/2
13255/13255 - 20s - loss: 58348.8047 - mse: 58348.8047 - val_loss: 40104.2930 - val_mse: 40104.2930
Epoch 2/2
13255/13255 - 20s - loss: 48545.0391 - mse: 48545.0391 - val_loss: 58635.6445 - val_mse: 58635.6445


In [9]:
# Evaluate the model
train_loss, train_mse = model.evaluate(X_train, y_train, verbose=1)
test_loss, test_mse = model.evaluate(X_val, y_val, verbose=1)

6197/6197 [==============================] - 4s 592us/step - loss: 58635.5430 - mse: 58635.5430


In [10]:
# Save the 
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_file_path = f'./files/models/model_{now}.h5py'
model.save(model_file_path)

INFO:tensorflow:Assets written to: ./files/models/model_20240702-215442.h5py\assets
